In [ ]:
from pathlib import Path

from typing import List
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from mothi.tiling_projects import QuPathTilingProject
from shapely.geometry import Point, Polygon
from shapely.geometry.base import BaseGeometry
from shapely.affinity import translate
import cv2

QP_Projectname = "comp_project"

In [ ]:
# create fig with ticks
def plot_image_mask(image: NDArray[np.int_], offset: float = 0.5,  *, label_dist: int = 5, tick_distance: int = 1) -> Axes:

    major_ticks_x = np.arange(0, image.shape[1], label_dist)
    minor_ticks_x = np.arange(0 + offset, image.shape[1], tick_distance)

    major_ticks_y = np.arange(0, image.shape[0], label_dist)
    minor_ticks_y = np.arange(0 + offset, image.shape[0], tick_distance)        

    fig, ax = plt.subplots()
    ax.set_xticks(major_ticks_x)
    ax.set_yticks(major_ticks_y)
    ax.set_xticks(minor_ticks_x, minor=True)
    ax.set_yticks(minor_ticks_y, minor=True)
    ax.grid(which='minor')
    ax.imshow(image)
    return ax

def overlay_image(img_ax: Axes, shape: BaseGeometry) -> Axes:
    offset_shape = translate(shape, -0.5, -0.5)
    img_ax.plot(*offset_shape.exterior.xy)
    return img_ax

In [ ]:
# list ground truth shapes
ground_truth_shapes: List[BaseGeometry] = [
    Point(6, 6).buffer(1),
    Polygon(((10.5,5.5),(15.5,5.5),(15.5,10.5),(10.5,10.5))),
    #Point(16,6).buffer(1.5),
    Point(21.5,6.5).buffer(2.5)
]

# get tile from QuPath with mothi
qp = QuPathTilingProject(QP_Projectname)
mask = qp.get_tile_annot_mask(0, (0,0), (25,12))
# plot tile mask
img_ax: Axes = plot_image_mask(mask, label_dist=2, tick_distance=1)
for shape in ground_truth_shapes:
    img_ax = overlay_image(img_ax, shape)
plt.show()

In [ ]:
groovy_im = cv2.imread('/workspaces/MOTH/comparison_groovy/groovy_export/white-32-labels.png')
groovy_im = np.array(cv2.cvtColor(groovy_im, cv2.COLOR_BGR2GRAY), dtype=np.int_)
groovy_ax = plot_image_mask(groovy_im)
for shape in ground_truth_shapes:
    groovy_ax = overlay_image(groovy_ax, shape)
plt.show()